# Pinecone VectorStore Async Lifecycle Demo

This notebook demonstrates two ways to manage the async Pinecone client that backs `PineconeVectorStore`:

1. Letting the store handle connections automatically by using `async with` (no manual close).
2. Closing the session yourself with `await store.aclose()` when you want deterministic cleanup.

Each section below walks through one of these approaches so you can run the cells and observe that the store no longer throws `RuntimeError: Session is closed` after back-to-back async calls.


## Prerequisites
- Install the project dependencies (see the repository README).
- Create or reuse a Pinecone serverless index that matches the dimensionality of your embeddings.
- Export the credentials before running: `export PINECONE_API_KEY=...` and optionally `export PINECONE_INDEX_NAME=...`.
- Provide an embedding model; this example uses `langchain-openai` but you can swap in any `Embeddings` implementation.

## Load Environment Variables
If you keep credentials in a `.env` file, the next cell loads it so you do not have to export environment variables manually each time. Feel free to skip this step if you prefer to set variables in your shell.


In [1]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
import os
import asyncio
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("Set the PINECONE_API_KEY environment variable before running this notebook.")

INDEX_NAME = os.environ.get("PINECONE_INDEX_NAME", "YOUR_INDEX_NAME")
if INDEX_NAME == "YOUR_INDEX_NAME":
    raise ValueError("Set PINECONE_INDEX_NAME or replace 'YOUR_INDEX_NAME' with an existing index name.")

embedding = OpenAIEmbeddings()

# Demo content to upsert and query
TEXTS = [
    "Pinecone is a vector database built for production workloads.",
    "LangChain integrates with Pinecone for semantic search use cases.",
    "Async workflows let you reuse Pinecone connections efficiently.",
]
METADATAS = [{"source": "demo", "idx": idx} for idx, _ in enumerate(TEXTS)]


### Demo Content
The snippets in `TEXTS` act as stand-ins for your own documents. Feel free to replace them with any list of strings and matching metadata before running the demos.


## Create Pinecone Index
A Pinecone index is a data structure that stores vector embeddings and allows for efficient similarity search. Before you can store or query embeddings, you need to create an index with the appropriate configuration (such as dimension and metric).

The following code will connect to Pinecone using your API key, check if an index with the specified name exists, and create it if necessary. This step is essential for managing and querying your vector data.

In [11]:
# Import Pinecone classes and utilities for index management
from pinecone import ServerlessSpec, Pinecone

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Define serverless deployment specification (cloud provider and region)
spec = ServerlessSpec(
    cloud="aws",
    region="us-west-2",  # You can change region as needed
)

In [5]:
# Optional: clean up a previous run by deleting the index.
# Uncomment the line below if you want to recreate the index from scratch.
# pc.delete_index(INDEX_NAME)


In [12]:
import time

index_name = INDEX_NAME
# List all existing indexes in your Pinecone project
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

# Check if the index already exists; if not, create it
if index_name not in existing_indexes:
    # Create a new index with specified dimension and metric
    pc.create_index(
        index_name,
        dimension=1536,  # Must match embedding output size
        metric="dotproduct",  # Similarity metric
        spec=spec,
    )
    # Wait for the index to be ready before proceeding
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

# Connect to the index for further operations
index = pc.Index(index_name)
time.sleep(1)
# View index statistics to confirm connection
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

## Scenario 1 – Use `async with` (no explicit close)
The context manager keeps a single client session open for the duration of the block and then closes it automatically when the block exits.


In [13]:
async def async_with_demo() -> None:
    """Demonstrate the async context manager to reuse a single Pinecone session."""
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embedding,
        namespace="async-demo",
    )

    print("Entering 'async with' block — a single aiohttp session backs all operations.")
    async with vectorstore:
        ids = await vectorstore.aadd_texts(TEXTS, metadatas=METADATAS)
        print(f"Upserted {len(ids)} vectors")

        print("First similarity search inside the context.")
        results = await vectorstore.asimilarity_search("pinecone async", k=2)
        for doc in results:
            print(f"  {doc.id}: {doc.page_content} -> {doc.metadata}")

        print("Second similarity search reuses the same session (no reconnect).")
        extra = await vectorstore.asimilarity_search("LangChain", k=1)
        for doc in extra:
            print(f"  {doc.id}: {doc.page_content} -> {doc.metadata}")

        await vectorstore.adelete(ids=ids)
        print("Cleaned up vectors while still inside the context.")

    print("Context exited — session closed automatically.")


In [14]:
# Run the async-context-manager demo
await async_with_demo()


Entering 'async with' block — a single aiohttp session backs all operations.
Upserted 3 vectors
First similarity search inside the context.
Second similarity search reuses the same session (no reconnect).
Cleaned up vectors while still inside the context.
Context exited — session closed automatically.


## Scenario 2 – Sequential calls without a context manager
Even without `async with`, the vector store now rebuilds the async client whenever a previous session has been closed. The loop below runs multiple add/search/delete cycles back-to-back without calling `aclose()` manually.


In [15]:
async def automatic_refresh_demo() -> None:
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embedding,
        namespace="auto-refresh-demo",
    )

    for run in range(2):
        print(f"Run {run + 1}: adding texts without an outer context manager.")
        payload = [f"Pinecone auto refresh demo {run}"]
        metadata = [{"source": "auto", "run": run}]
        ids = await vectorstore.aadd_texts(payload, metadatas=metadata)
        print(f"  Upserted ids: {ids}")

        results = await vectorstore.asimilarity_search("pinecone", k=1)
        for doc in results:
            print(f"  Search hit: {doc.page_content} -> {doc.metadata}")

        await vectorstore.adelete(ids=ids)
        print("  Deleted vectors; the next loop iteration will reopen a fresh session automatically.")

    print("Finished sequential runs without ever calling aclose().")


In [16]:
# Run the sequential demo without an explicit context manager
await automatic_refresh_demo()


Run 1: adding texts without an outer context manager.
  Upserted ids: ['62ac61ed-f228-4514-8925-60a54d090d24']
  Deleted vectors; the next loop iteration will reopen a fresh session automatically.
Run 2: adding texts without an outer context manager.
  Upserted ids: ['c177be63-b003-4fe3-9c39-5138524dbf20']
  Deleted vectors; the next loop iteration will reopen a fresh session automatically.
Finished sequential runs without ever calling aclose().


## Scenario 3 – Explicitly close the async session
Call `await store.aclose()` when you want deterministic cleanup after a set of operations (for example before handing the store to another task). The store can still be reused afterwards because it will lazily build a new session on the next async call.


In [17]:
async def manual_close_demo() -> None:
    """Show explicit lifecycle management with aclose()."""
    vectorstore = PineconeVectorStore(
        index_name=INDEX_NAME,
        embedding=embedding,
        namespace="manual-demo",
    )

    ids = await vectorstore.aadd_texts(TEXTS[:1], metadatas=METADATAS[:1])
    print(f"Added initial ids: {ids}")
    try:
        results = await vectorstore.asimilarity_search("pinecone", k=1)
        for doc in results:
            print(f"  Search hit: {doc.page_content} -> {doc.metadata}")
    finally:
        await vectorstore.adelete(ids=ids)
        print("Deleted initial vectors; calling aclose() to release the session.")
        await vectorstore.aclose()

    print("Session closed. The next call recreates the client lazily.")
    follow_up_metadata = [{"source": "manual", "stage": "follow-up"}]
    follow_up_ids = await vectorstore.aadd_texts(TEXTS[1:2], metadatas=follow_up_metadata)
    print(f"Added follow-up ids: {follow_up_ids}")
    try:
        follow_up_results = await vectorstore.asimilarity_search("langchain", k=1)
        for doc in follow_up_results:
            print(f"  Follow-up hit: {doc.page_content} -> {doc.metadata}")
    finally:
        await vectorstore.adelete(ids=follow_up_ids)
        await vectorstore.aclose()
        print("Explicit close called again to tidy up.")


In [18]:
# Run the explicit close demo
await manual_close_demo()


Added initial ids: ['3420a3ab-22d4-456a-8f70-340f29383896']
Deleted initial vectors; calling aclose() to release the session.
Session closed. The next call recreates the client lazily.
Added follow-up ids: ['0fb7b67d-15b2-4dfd-b4cf-e6c248f845f1']
Explicit close called again to tidy up.


## Notes
- Replace the demo texts and metadata with your own dataset to mirror production behaviour.
- `async with PineconeVectorStore(...)` keeps one HTTP session open across the block and closes it automatically on exit.
- Without an `async with` block, each call now reinitialises the session whenever the previous one has been closed, so you can run back-to-back async operations safely.
- `await store.aclose()` is still useful when you want deterministic cleanup between batches or before handing the store to another component.
- If you run these cells multiple times, consider changing the namespace or cleaning up vectors to avoid duplicate data.
